<a href="https://colab.research.google.com/github/AlexeyTri/PyTorchTutorials/blob/main/Tutorial7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataLoader

In [78]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math

gradient computation etc. not efficient for whole data set

divide dataset into small batches

---
trainig loop

    for epoch in range(num_epochs):

        <!-- loop over all batches -->
        for i in range(total_batches):
            batch_x, batch_y = ...

* epoch  - one forward and backward pass of ALL training sample
* batch_size - number of training samples used in one forward/backward pass
* number of iterations - number od passes, each pass (forward+backward) using [batch_size] number of samples
* e.g.: 100 samples, batch_size=20 => 100/20=5 iterations for 1 epoch

**Dataloader can do the batch computation fot us:**

implement a custom Dataset:

inherit Dataset

implement __init__, __getitem__, __len__


In [64]:
# Clone the entire repo.
!git clone -l -s https://github.com/AlexeyTri/PyTorchTutorials.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 29 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (29/29), done.
/content/cloned-repo
README.md	 Tutorial2.ipynb  Tutorial4.ipynb  Tutorial6.ipynb
Tutorial1.ipynb  Tutorial3.ipynb  Tutorial5.ipynb  wine.csv


In [65]:
class WineDataset(Dataset):
    def __init__(self):
        # initialize data, download, etc.
        # read with numpy or pandas
        xy = np.loadtxt('/content/cloned-repo/wine.csv', delimiter=',',dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]

        # here we first column is the class label, rest are the features
        self.x_data = torch.from_numpy(xy[:, 1:])
        self.y_data = torch.from_numpy(xy[:, [0]])

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [75]:
# create dataset
dataset = WineDataset()

In [76]:
# get first sample and unpack
first_data = dataset[0]
features, labels = first_data
features, labels

(tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
         3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
         1.0650e+03]), tensor([1.]))

In [80]:
# load whole dataset with Dataloader
# shuffle: shuffle data, good for trainig
# num_workers: faster loading with multiple subprocesses
# if you get an error during loading, set num_workers to 0
train_loader = DataLoader(dataset=dataset,
                        batch_size=1,
                        shuffle=True,
                        num_workers=2)

In [81]:
# convert to an iterator and look at one random sample
dataiter = iter(train_loader)
data = next(dataiter)
features, labels = data
features, labels

(tensor([[1.3880e+01, 5.0400e+00, 2.2300e+00, 2.0000e+01, 8.0000e+01, 9.8000e-01,
          3.4000e-01, 4.0000e-01, 6.8000e-01, 4.9000e+00, 5.8000e-01, 1.3300e+00,
          4.1500e+02]]), tensor([[3.]]))

In [ ]:
# Dummy training loop
num_epochs = 2
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/4)
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # here: 178 samples, batch_size=4, n_iters=178/4=44.5 -> 45 iterations
        if (i+1) %5 == 0:
            print(f'Epoch: {epoch+1}/{num_epochs}, Step {i+1}/{n_iterations}| Inputs {inputs.shape} | Labels {labels.shape}')

In [ ]:
# some famous datasets are available in torchvision.datasets
# e.g. MNIST, Fashion-MNIST, CIFAR10, COCO

train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=torchvision.transforms.ToTensor(),  
                                           download=True)

train_loader = DataLoader(dataset=train_dataset, 
                                           batch_size=3, 
                                           shuffle=True)

In [84]:
# look at one random sample
dataiter = iter(train_loader)
data = next(dataiter)
inputs, targets = data
inputs.shape, targets.shape

(torch.Size([3, 1, 28, 28]), torch.Size([3]))